This notebook compares the scores of the local models and the model chain

In [39]:
import numpy as np
import pandas as pd
#import itertools
#import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error as mse, brier_score_loss
from chaining import Chain
import os

In [40]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [41]:
def missingness_and_categorical_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Stratify categorical variables
    for col in df.select_dtypes(include=['category']):
        counts = df[col].value_counts(normalize=True)
        for category in counts.index:
            idx = df[col] == category
            cv[idx] = cv[idx].fillna(np.random.choice(np.where(idx)[0], size=int(counts[category] * N_FOLDS), replace=False))

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [42]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

data = pd.read_csv(path)

# Bin the number of relapses into 0, 1, 2, 3 and 4+ 
def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)
data

,USUBJID,AGE,SEX,RACE,CONTINENT,CESEV,CECONTRT,TOTRELAP,MHCONTRT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,SMSTDY,NRELAP,NHPT-before,NHPT-2y,NHPT-after_2y,PASAT_2s-before,PASAT_2s-2y,PASAT_2s-after_2y,PASAT_3s-before,PASAT_3s-2y,PASAT_3s-after_2y,SDMT-before,SDMT-2y,T25FW-before,T25FW-2y,T25FW-after_2y,T-before,T-after,P-before,P-after,N-before,N-after,SLEC_before,SLEC_after,SES_after,SES_before,VAA,BDI-before,BDI-after,EDSS-before,EDSS-2y,EDSS-after_2y,KFSS1-Sensory-2y,KFSS1-Sensory-after_2y,KFSS1-Sensory-before,KFSS1-Brain-2y,KFSS1-Brain-after_2y,KFSS1-Brain-before,KFSS1-Bowel-2y,KFSS1-Bowel-after_2y,KFSS1-Bowel-before,KFSS1-Pyramidal-2y,KFSS1-Pyramidal-after_2y,KFSS1-Pyramidal-before,KFSS1-Cerebral-2y,KFSS1-Cerebral-after_2y,KFSS1-Cerebral-before,KFSS1-Visual-2y,KFSS1-Visual-after_2y,KFSS1-Visual-before,KFSS1-Cerebellar-2y,KFSS1-Cerebellar-after_2y,KFSS1-Cerebellar-before,KFSS_M-2y,KFSS_M-after_2y,KFSS_M-before,KFSS_P-2y,KFSS_P-after_2y,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,M_R36-SF12-after,P_R36-SF12-after,R36-SF12-after_Ind
0,MSOAC/0014,46.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSOAC/0016,NaN,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,Y,SPMS,1,1,0,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.55,6.60,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSOAC/0019,44.0,M,NON-WHITE,NaN,NaN,NaN,NaN,NaN,PPMS,1,1,0,0,NaN,0.0,23.65,21.30,20.15,34.5,35.5,43.0,43.5,51.0,53.0,NaN,NaN,6.30,6.15,5.85,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,3.50,3.0,0.333333,0.166667,0.500000,0.2,0.0,0.2,0.000000,0.166667,0.083333,0.333333,0.5,0.416667,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.2,0.0,0.185185,0.185185,0.240741,0.166667,0.083333,0.208333,0.828571,0.772152,1.0,0.857143,0.721519,1.0
3,MSOAC/0024,60.0,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,NaN,SPMS,1,1,1,1,NaN,0.0,34.45,37.50,NaN,55.0,54.0,NaN,60.0,60.0,NaN,NaN,NaN,4.50,5.25,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.031746,0.023810,4.00,3.75,NaN,0.333333,NaN,0.333333,0.0,NaN,0.1,0.583333,NaN,0.666667,0.166667,NaN,0.250000,0.0,NaN,0.0,0.000000,NaN,0.083333,0.2,NaN,0.5,0.129630,NaN,0.240741,0.291667,NaN,0.375000,0.885714,0.569620,1.0,0.857143,0.716216,1.0
4,MSOAC/0030,28.0,F,WHITE,EUROPE,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,16.55,17.90,NaN,NaN,NaN,NaN,58.0,60.0,NaN,63.5,69.0,4.85,4.70,NaN,0.0,0.0,0.0,0.0,0.0,0.0,26.0,24.0,1.25,1.25,NaN,0.063492,0.039683,2.00,1.50,NaN,0.166667,NaN,0.166667,0.2,NaN,0.2,0.166667,NaN,0.166667,0.166667,NaN,0.333333,0.0,NaN,0.2,0.166667,NaN,0.083333,0.0,NaN,0.1,0.111111,NaN,0.203704,0.166667,NaN,0.125000,0.933333,0.846154,0.0,0.833333,0.730769,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,MSOAC/9986,46.0,M,WHITE,OCEANIA,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,19.35,18.95,NaN,NaN,NaN,NaN,58.0,60.0,NaN,51.0,60.0,3.90,3.80,NaN,0.0,0.0,0.0,0.0,0.0,0.0,36.0,35.0,1.25,1.25,NaN,0.047619,0.063492,2.75,2.50,NaN,0.333333,NaN,0.166667,0.0,NaN,0.0,0.333333,NaN,0.250000,0.166667,NaN,0.333333,0.0,NaN,0.0,0.000000,NaN,0.000000,0.0,NaN,0.2,0.111111,NaN,0.148148,0.166667,NaN,0.125000,0.833333,0.730769,0.0,0.800000,0.750000,0.0
2461,MSOAC/9987,18.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [43]:
#variables = ['KFSS_M-2y', 'EDSS-2y', 'T25FW-2y', 'NRELAP']# removed KFSS_P-2y for now -- ('SMSTDY' gave a score of -0.03)
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'EDSS-after_2y', 'NRELAP', 'CESEV']

Note: once we obtain the best ordering, change the order here!

In [44]:
# Extract targets
targets = data[variables]

# Extract features by dropping the target columns
#features = data.drop(variables, axis=1)

columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'NHPT-before', 'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'T25FW-before', 'SLEC_before','SES_before',
                    'BDI-before', 'EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before',
                	'P_R36-SF12-before', 'R36-SF12-before_Ind', 'T-before','P-before','N-before']
# still need to change in OE dataframe the SLEC and SES so name is consistent with the others

features = data[columns_to_keep]
features

,AGE,SEX,RACE,CONTINENT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before
0,46.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,M,WHITE,NORTH AMERICA,SPMS,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,44.0,M,NON-WHITE,NaN,PPMS,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN
3,60.0,M,WHITE,NORTH AMERICA,SPMS,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0
4,28.0,F,WHITE,EUROPE,RRMS,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.0,M,WHITE,OCEANIA,RRMS,1,1,0,1,19.35,NaN,58.0,51.0,3.90,36.0,1.25,0.047619,2.75,0.148148,0.125000,0.833333,0.730769,0.0,0.0,0.0,0.0
2461,18.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2462,38.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2463,40.0,F,WHITE,NaN,PPMS,0,1,0,1,23.80,21.5,31.5,NaN,6.15,NaN,NaN,NaN,4.50,0.481481,0.166667,0.728571,0.658228,1.0,0.0,0.0,NaN


In [45]:
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
features.head()

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS
0,46.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,1,0
1,NaN,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,1,0,1,0,0,0,1,0,0,0,0,1
2,44.0,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN,0,1,1,0,0,0,0,0,0,0,1,0,0
3,60.0,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0,0,1,0,1,0,0,0,1,0,0,0,0,1
4,28.0,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [46]:
targets.dtypes

KFSS_M-2y           float64
KFSS_P-2y           float64
EDSS-2y             float64
T25FW-2y            float64
NHPT-2y             float64
P_R36-SF12-after    float64
M_R36-SF12-after    float64
SES_after           float64
SLEC_after          float64
KFSS_M-after_2y     float64
KFSS_P-after_2y     float64
EDSS-after_2y       float64
NRELAP               object
CESEV                object
dtype: object

Run MICE

In [47]:
featuresM=features.copy()

#missing_mask = featuresM.isna()
imputer = IterativeImputer(max_iter=10, random_state=42)
imputed_values = imputer.fit_transform(featuresM)

featuresM = pd.DataFrame(imputed_values, columns=featuresM.columns)
featuresM

c:\Users\lenne\anaconda3\envs\Thesis\lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS
0,46.000000,0.0,0.0,0.0,0.0,25.018065,34.403347,43.574182,43.707050,10.321359,16.207326,0.953592,0.266973,3.302396,0.199786,0.111704,0.576803,0.636868,0.822802,0.013470,0.017288,0.019983,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,49.999556,1.0,1.0,0.0,1.0,32.248395,34.728405,45.174473,42.865450,8.550000,16.466525,0.994179,0.137418,6.000000,0.408692,0.302219,0.713367,0.593077,0.812958,0.000000,0.022124,0.030678,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,44.000000,1.0,1.0,0.0,0.0,23.650000,34.500000,43.500000,34.041493,6.300000,28.946702,0.992250,0.137595,3.750000,0.240741,0.208333,0.828571,0.772152,1.000000,0.000000,0.000000,0.016297,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,60.000000,1.0,1.0,1.0,1.0,34.450000,55.000000,60.000000,147.920671,4.500000,-79.622500,1.060940,0.031746,4.000000,0.240741,0.375000,0.885714,0.569620,1.000000,0.000000,0.000000,1.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,28.000000,1.0,1.0,0.0,1.0,16.550000,46.775832,58.000000,63.500000,4.850000,26.000000,1.250000,0.063492,2.000000,0.203704,0.125000,0.933333,0.846154,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.000000,1.0,1.0,0.0,1.0,19.350000,46.037596,58.000000,51.000000,3.900000,36.000000,1.250000,0.047619,2.750000,0.148148,0.125000,0.833333,0.730769,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2461,18.000000,0.0,0.0,0.0,0.0,24.024269,35.002518,44.138024,51.017268,10.424487,21.483848,1.003534,0.299753,3.036980,0.183260,0.104252,0.557752,0.669654,0.773500,0.005435,0.008969,0.007250,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2462,38.000000,0.0,0.0,0.0,0.0,24.734123,34.574539,43.735280,45.795684,10.350824,17.714904,0.967861,0.276339,3.226563,0.195064,0.109575,0.571360,0.646235,0.808716,0.011174,0.014911,0.016345,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2463,40.000000,0.0,1.0,0.0,1.0,23.800000,21.500000,31.500000,14.910542,6.150000,49.104452,0.971514,0.248296,4.500000,0.481481,0.166667,0.728571,0.658228,1.000000,0.000000,0.000000,0.017645,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [48]:
selected_columns = featuresM.iloc[:, :-2]

# Compute the range for each column
ranges = selected_columns.apply(lambda x: x.max() - x.min())

print("Range of values for each column (except last two):")
print(ranges)

Range of values for each column (except last two):
AGE                         65.055452
CARDIO                       1.000000
URINARY                      1.000000
MUSCKELET                    1.000000
FATIGUE                      1.000000
NHPT-before                288.400000
PASAT_2s-before             67.470180
PASAT_3s-before             58.500000
SDMT-before                599.509092
T25FW-before               131.400000
SLEC_before                676.739216
SES_before                   1.195100
BDI-before                   0.856293
EDSS-before                  6.500000
KFSS_M-before                0.685185
KFSS_P-before                0.750000
M_R36-SF12-before            0.885714
P_R36-SF12-before            0.769231
R36-SF12-before_Ind          1.292154
T-before                     1.004456
P-before                     1.001466
N-before                     1.009040
SEX_F                        1.000000
SEX_M                        1.000000
RACE_NON-WHITE               1.000000

In [49]:
model_data = pd.concat([featuresM, targets], axis=1)
model_data

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS,KFSS_M-2y,KFSS_P-2y,EDSS-2y,T25FW-2y,NHPT-2y,P_R36-SF12-after,M_R36-SF12-after,SES_after,SLEC_after,KFSS_M-after_2y,KFSS_P-after_2y,EDSS-after_2y,NRELAP,CESEV
0,46.000000,0.0,0.0,0.0,0.0,25.018065,34.403347,43.574182,43.707050,10.321359,16.207326,0.953592,0.266973,3.302396,0.199786,0.111704,0.576803,0.636868,0.822802,0.013470,0.017288,0.019983,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,49.999556,1.0,1.0,0.0,1.0,32.248395,34.728405,45.174473,42.865450,8.550000,16.466525,0.994179,0.137418,6.000000,0.408692,0.302219,0.713367,0.593077,0.812958,0.000000,0.022124,0.030678,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,6.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,44.000000,1.0,1.0,0.0,0.0,23.650000,34.500000,43.500000,34.041493,6.300000,28.946702,0.992250,0.137595,3.750000,0.240741,0.208333,0.828571,0.772152,1.000000,0.000000,0.000000,0.016297,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.185185,0.166667,3.50,6.15,21.30,0.721519,0.857143,NaN,NaN,0.185185,0.083333,3.0,0.0,NaN
3,60.000000,1.0,1.0,1.0,1.0,34.450000,55.000000,60.000000,147.920671,4.500000,-79.622500,1.060940,0.031746,4.000000,0.240741,0.375000,0.885714,0.569620,1.000000,0.000000,0.000000,1.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.129630,0.291667,3.75,5.25,37.50,0.716216,0.857143,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,28.000000,1.0,1.0,0.0,1.0,16.550000,46.775832,58.000000,63.500000,4.850000,26.000000,1.250000,0.063492,2.000000,0.203704,0.125000,0.933333,0.846154,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.111111,0.166667,1.50,4.70,17.90,0.730769,0.833333,1.25,24.0,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.000000,1.0,1.0,0.0,1.0,19.350000,46.037596,58.000000,51.000000,3.900000,36.000000,1.250000,0.047619,2.750000,0.148148,0.125000,0.833333,0.730769,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.111111,0.166667,2.50,3.80,18.95,0.750000,0.800000,1.25,35.0,NaN,NaN,NaN,0.0,NaN
2461,18.000000,0.0,0.0,0.0,0.0,24.024269,35.002518,44.138024,51.017268,10.424487,21.483848,1.003534,0.299753,3.036980,0.183260,0.104252,0.557752,0.669654,0.773500,0.005435,0.008969,0.007250,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2462,38.000000,0.0,0.0,0.0,0.0,24.734123,34.574539,43.735280,45.795684,10.350824,17.714904,0.967861,0.276339,3.226563,0.195064,0.109575,0.571360,0.646235,0.808716,0.011174,0.014911,0.016345,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,MILD
2463,40.000000,0.0,1.0,0.0,1.0,23.800000,21.500000,31.500000,14.910542,6.150000,49.104452,0.971514,0.248296,4.500000,0.481481,0.166667,0.728571,0.658228,1.000000,0.000000,0.000000,0.017645,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.314815,0.166667,3.75,6.00,22.40,0.594937,0.757143,NaN,NaN,0.351852,0.083333,4.0,1.0,NaN


5-Fold CV

In [50]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [51]:
# Generate CV folds
cv=missingness_and_categorical_stratified_cv(targets, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

featuresM_cv = pd.merge(featuresM, pd.DataFrame(cv), left_index=True, right_index=True)
targets_cv = pd.merge(targets, pd.DataFrame(cv), left_index=True, right_index=True)

featuresM_cv['CV Fold'].value_counts()

CV Fold
4.0    510
3.0    502
0.0    500
1.0    495
2.0    458
Name: count, dtype: int64

In [52]:
def normalized_mean_squared_error(true, pred, train):
    num = mse(true, pred)
    mean_value = np.mean(train)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

---

### Local Models

In [53]:
y_pred_list = []
y_test_list = []
y_train_list = []
y_pred_prob_list = []
yi_test_dummies_list = []
yi_train_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))
    y_train_list.append(pd.DataFrame(yi_train, columns=yi_train.columns, index=yi_train.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    subset_yi_train = yi_train.select_dtypes(include=['object'])
    yi_train_dummies = pd.get_dummies(subset_yi_train, columns=subset_yi_train.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate=False, #RUN LOCAL MODELS 
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    yi_train_dummies_list.append(yi_train_dummies)
    
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [54]:
# Obtain values of baseline model
yi_train_dummies_avg = []
i=0
 
for yi_train_dummies_fold in yi_train_dummies_list:

    percentages = yi_train_dummies_fold.sum() / len(yi_train_dummies_fold)

    yi_train_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_list[i].index, columns=yi_train_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_train_dummies_avg_fold.columns:
        yi_train_dummies_avg_fold[col] = yi_train_dummies_avg_fold[col].apply(lambda x: percentages[col])
    
    i += 1
    yi_train_dummies_avg.append(yi_train_dummies_avg_fold)

In [55]:
# Re-arrange output of predited probabilities
concatenated_dfs = []

for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):

        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

In [56]:
# Initialize a list to store scores
scores_with_std = []
avg_brier_score = []
avg_baseline_score = []
variables_cat = yi_test_dummies_list[0].columns
cat_normalized_brier=[]

# Create a dictionary to store the scores for variables with the same letters before the '_'
brier_scores_dict = {}
baseline_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    brier_scores = []
    baseline_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][level_name] 
        
        #N=len(y_test)
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        N_brier_score = brier_score#*N
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        N_brier_baseline = brier_baseline#*N
        #normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        brier_scores.append(N_brier_score)
        baseline_scores.append(N_brier_baseline)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in brier_scores_dict:
        brier_scores_dict[prefix].extend(brier_scores)
    else:
        brier_scores_dict[prefix] = brier_scores

    if prefix in baseline_scores_dict:
        baseline_scores_dict[prefix].extend(baseline_scores)
    else:
        baseline_scores_dict[prefix] = baseline_scores


# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in brier_scores_dict.items():
    sum_score = np.sum(scores)
    #avg_score = np.mean(scores)
    avg_brier_score.append((prefix, sum_score))

for prefix, scores in baseline_scores_dict.items():
    sum_score = np.sum(scores)
    #avg_score = np.mean(scores)
    avg_baseline_score.append((prefix, sum_score))


normalized_score_list = []
for i in range(len(avg_brier_score)):
    normalized_score = avg_brier_score[i][1]/avg_baseline_score[i][1]
    cell = (avg_brier_score[i][0], normalized_score)
    normalized_score_list.append(cell)

#cat_normalized_brier = []
#cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score in normalized_score_list:
    print(f"{prefix}: {avg_score:.2f} ")
    cat_normalized_brier.append(avg_score)

Normalized Brier scores for each categorical variable:
NRELAP: 0.97 
CESEV: 1.69 


In [57]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)

In [58]:
# Obtain normalized MSE
scores_with_std = []

for variable_name in variables: 
    variable_scores = []
    
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            y_train = y_train_list[fold_index][variable_name]  

            score = normalized_mean_squared_error(y_test, y_pred, y_train)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (local):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (local):
KFSS_M-2y: 0.20 (± 0.02)
KFSS_P-2y: 0.25 (± 0.04)
EDSS-2y: 0.11 (± 0.01)
T25FW-2y: 0.28 (± 0.08)
NHPT-2y: 0.53 (± 0.14)
P_R36-SF12-after: 0.31 (± 0.02)
M_R36-SF12-after: 0.43 (± 0.03)
SES_after: 0.31 (± 0.06)
SLEC_after: 0.33 (± 0.04)
KFSS_M-after_2y: 0.34 (± 0.05)
KFSS_P-after_2y: 0.47 (± 0.06)
EDSS-after_2y: 0.23 (± 0.02)


In [59]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19618477, 0.24767953, 0.11433688, 0.27662151, 0.52776057,
       0.30705999, 0.43374968, 0.31476637, 0.33426621, 0.33561124,
       0.47413775, 0.23448327, 0.96835378, 1.68615001])

In [60]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Normalized unifying score:", average_normalized_brier)

Normalized unifying score: 0.4607972529914672


### Propagate predictions

In [61]:
y_pred_list = []
y_test_list = []
y_train_list = []
y_pred_prob_list = []
yi_test_dummies_list = []
yi_train_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))
    y_train_list.append(pd.DataFrame(yi_train, columns=yi_train.columns, index=yi_train.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    subset_yi_train = yi_train.select_dtypes(include=['object'])
    yi_train_dummies = pd.get_dummies(subset_yi_train, columns=subset_yi_train.columns, dtype=int)


    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="pred", #RUN MODELS IN A CHAIN
    )


    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    yi_train_dummies_list.append(yi_train_dummies)
    
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [62]:
yi_train_dummies_avg = []
i=0
 
for yi_train_dummies_fold in yi_train_dummies_list:

    percentages = yi_train_dummies_fold.sum() / len(yi_train_dummies_fold)

    yi_train_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_list[i].index, columns=yi_train_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_train_dummies_avg_fold.columns:
        yi_train_dummies_avg_fold[col] = yi_train_dummies_avg_fold[col].apply(lambda x: percentages[col])
    
    i += 1
    yi_train_dummies_avg.append(yi_train_dummies_avg_fold)

In [63]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

In [64]:
# Initialize a list to store scores
scores_with_std = []
avg_brier_score = []
avg_baseline_score = []
variables_cat = yi_test_dummies_list[0].columns
cat_normalized_brier=[]

# Create a dictionary to store the scores for variables with the same letters before the '_'
brier_scores_dict = {}
baseline_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    brier_scores = []
    baseline_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][level_name] 
        
        #N=len(y_test)
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        N_brier_score = brier_score#*N
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        N_brier_baseline = brier_baseline#*N
        #normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        brier_scores.append(N_brier_score)
        baseline_scores.append(N_brier_baseline)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in brier_scores_dict:
        brier_scores_dict[prefix].extend(brier_scores)
    else:
        brier_scores_dict[prefix] = brier_scores

    if prefix in baseline_scores_dict:
        baseline_scores_dict[prefix].extend(baseline_scores)
    else:
        baseline_scores_dict[prefix] = baseline_scores


# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in brier_scores_dict.items():
    sum_score = np.sum(scores)
    #avg_score = np.mean(scores)
    avg_brier_score.append((prefix, sum_score))

for prefix, scores in baseline_scores_dict.items():
    sum_score = np.sum(scores)
    #avg_score = np.mean(scores)
    avg_baseline_score.append((prefix, sum_score))


normalized_score_list = []
for i in range(len(avg_brier_score)):
    normalized_score = avg_brier_score[i][1]/avg_baseline_score[i][1]
    cell = (avg_brier_score[i][0], normalized_score)
    normalized_score_list.append(cell)

#cat_normalized_brier = []
#cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score in normalized_score_list:
    print(f"{prefix}: {avg_score:.2f} ")
    cat_normalized_brier.append(avg_score)

Normalized Brier scores for each categorical variable:
NRELAP: 0.97 
CESEV: 1.69 


In [65]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)


# Compute normalized MSE
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            y_train = y_train_list[fold_index][variable_name]

            score = normalized_mean_squared_error(y_test, y_pred, y_train)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - propagate predictions):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - propagate predictions):
KFSS_M-2y: 0.20 (± 0.02)
KFSS_P-2y: 0.25 (± 0.04)
EDSS-2y: 0.12 (± 0.01)
T25FW-2y: 0.26 (± 0.08)
NHPT-2y: 0.49 (± 0.14)
P_R36-SF12-after: 0.30 (± 0.02)
M_R36-SF12-after: 0.45 (± 0.05)
SES_after: 0.31 (± 0.05)
SLEC_after: 0.34 (± 0.04)
KFSS_M-after_2y: 0.35 (± 0.05)
KFSS_P-after_2y: 0.49 (± 0.04)
EDSS-after_2y: 0.25 (± 0.02)


In [66]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19618477, 0.24947923, 0.11777731, 0.25806906, 0.49355363,
       0.30351181, 0.45069556, 0.30941555, 0.33831107, 0.34990521,
       0.49062596, 0.24594276, 0.96987424, 1.69113026])

In [67]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Normalized unifying score:", average_normalized_brier)

Normalized unifying score: 0.46174831568119395


### Propagate true values

In [68]:
y_pred_list = []
y_test_list = []
y_train_list = []
y_pred_prob_list = []
yi_test_dummies_list = []
yi_train_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))
    y_train_list.append(pd.DataFrame(yi_train, columns=yi_train.columns, index=yi_train.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    subset_yi_train = yi_train.select_dtypes(include=['object'])
    yi_train_dummies = pd.get_dummies(subset_yi_train, columns=subset_yi_train.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="true", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    yi_train_dummies_list.append(yi_train_dummies)
    
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [69]:
yi_train_dummies_avg = []
i=0

for yi_train_dummies_fold in yi_train_dummies_list:

    percentages = yi_train_dummies_fold.sum() / len(yi_train_dummies_fold)

    yi_train_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_list[i].index, columns=yi_train_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_train_dummies_avg_fold.columns:
        yi_train_dummies_avg_fold[col] = yi_train_dummies_avg_fold[col].apply(lambda x: percentages[col])
    
    i += 1
    yi_train_dummies_avg.append(yi_train_dummies_avg_fold)

In [70]:
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

In [71]:
# Initialize a list to store scores
scores_with_std = []
avg_brier_score = []
avg_baseline_score = []
variables_cat = yi_test_dummies_list[0].columns
cat_normalized_brier=[]

# Create a dictionary to store the scores for variables with the same letters before the '_'
brier_scores_dict = {}
baseline_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    brier_scores = []
    baseline_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][level_name] 
        
        #N=len(y_test)
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        N_brier_score = brier_score#*N
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        N_brier_baseline = brier_baseline#*N
        #normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        brier_scores.append(N_brier_score)
        baseline_scores.append(N_brier_baseline)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in brier_scores_dict:
        brier_scores_dict[prefix].extend(brier_scores)
    else:
        brier_scores_dict[prefix] = brier_scores

    if prefix in baseline_scores_dict:
        baseline_scores_dict[prefix].extend(baseline_scores)
    else:
        baseline_scores_dict[prefix] = baseline_scores


# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in brier_scores_dict.items():
    sum_score = np.sum(scores)
    #avg_score = np.mean(scores)
    avg_brier_score.append((prefix, sum_score))

for prefix, scores in baseline_scores_dict.items():
    sum_score = np.sum(scores)
    #avg_score = np.mean(scores)
    avg_baseline_score.append((prefix, sum_score))


normalized_score_list = []
for i in range(len(avg_brier_score)):
    normalized_score = avg_brier_score[i][1]/avg_baseline_score[i][1]
    cell = (avg_brier_score[i][0], normalized_score)
    normalized_score_list.append(cell)

#cat_normalized_brier = []
#cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score in normalized_score_list:
    print(f"{prefix}: {avg_score:.2f} ")
    cat_normalized_brier.append(avg_score)

Normalized Brier scores for each categorical variable:
NRELAP: 0.95 
CESEV: 1.62 


In [72]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)

In [73]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            y_train = y_train_list[fold_index][variable_name]
            
            score = normalized_mean_squared_error(y_test, y_pred, y_train)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.20 (± 0.02)
KFSS_P-2y: 0.25 (± 0.03)
EDSS-2y: 0.12 (± 0.01)
T25FW-2y: 0.27 (± 0.08)
NHPT-2y: 0.47 (± 0.12)
P_R36-SF12-after: 0.30 (± 0.02)
M_R36-SF12-after: 0.45 (± 0.04)
SES_after: 0.31 (± 0.05)
SLEC_after: 0.34 (± 0.04)
KFSS_M-after_2y: 0.34 (± 0.04)
KFSS_P-after_2y: 0.49 (± 0.03)
EDSS-after_2y: 0.23 (± 0.03)


In [74]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19618477, 0.24937854, 0.11522405, 0.27127313, 0.46509804,
       0.29885761, 0.44659859, 0.31153114, 0.33961115, 0.34033497,
       0.48661452, 0.23338493, 0.9544289 , 1.61973377])

In [75]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Normalized unifying score:", average_normalized_brier)

Normalized unifying score: 0.45201815062600026
